#Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [1]:
from crewai import Agent, Crew, Task

In [3]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# Ensure your SerperDev API key is set in the environment variables
os.environ["SERPER_API_KEY"] = os.environ["SERPER_DEV_API_KEY"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## crewAI Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/ashish/Desktop/vettura-genai/Codes/genai_env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## Creating Agents

In [4]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [12]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [13]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [7]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [14]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [15]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=False,
    agent=logistics_manager
)

In [24]:
marketing_task = Task(
    description="Promote the {event_topic} to be held on {tentative_date} in the event location finalized by Venue Coordinator"
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [25]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

2025-02-14 10:01:26,694 - 7955664896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [26]:
print(event_management_crew)

name=None cache=True tasks=[Task(description=Find a venue in Bengaluru that meets criteria for VLSI Design Conference., expected_output=All the details of a specifically chosenvenue you found to accommodate the event.), Task(description=Coordinate catering and equipment for an event with 500 participants on 2025-Mar-20., expected_output=Confirmation of all logistics arrangements including catering and equipment setup.), Task(description=Promote the {event_topic} aiming to engage at least{expected_participants} potential attendees., expected_output=Report on marketing activities and attendee engagement formatted as markdown.)] agents=[Agent(role=Venue Coordinator, goal=Identify and book an appropriate venue based on event requirements, backstory=With a keen sense of space and understanding of event logistics, you excel at finding and securing the perfect venue that fits the event's theme, size, and budget constraints.), Agent(role=Logistics Manager, goal=Manage all logistics for the eve

## Running the Crew

- Set the inputs for the execution of the crew.

In [27]:
event_details = {
    'event_topic': "VLSI Design Conference",
    'event_description': "A gathering of academicians "
                         "and industry leaders "
                         "to present current research and commercial applications in VLSI design.",
    'event_city': "Bengaluru",
    'tentative_date': "2025-Mar-20",
    'expected_participants': 500,
    'budget': 2000000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**:
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [28]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in Bengaluru that meets criteria for VLSI Design Conference.


# Agent: Venue Coordinator
## Thought: I need to search for venues in Bengaluru that can accommodate a VLSI Design Conference, considering their size, amenities, and location.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"VLSI Design Conference venue Bengaluru\"}"
## Tool Output: 
{'searchParameters': {'q': 'VLSI Design Conference venue Bengaluru', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'VLSID 2025 | International Conference on VLSI Design ...', 'link': 'https://vlsid.org/', 'snippet': 'International Conference on VLSI Design & Embedded Systems | VLSID 2025. ... User Design Track · Design Contest · Fellowship. Forums. Student Research Forum / PhD ...', 'position': 1, 'sitelinks': [{'title': 'Accomodation', 'link': 'https://vlsid.org/accomodation/'}, {'title': 'Technical Program Committee', 'link': 'https:

- Display the generated `venue_details.json` file.

In [29]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'The Leela Palace, 23, Kodihalli, Bangalore - 560008, Karnataka, '
            'India',
 'booking_status': 'Available for booking, please contact for details.',
 'capacity': 2400,
 'name': 'The Leela Palace Bengaluru'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [30]:
from IPython.display import Markdown
Markdown(r"marketing_report.md")

```markdown
# VLSI Design Conference 2025 Promotion

## Event Details
- **Name:** VLSI Design Conference 2025
- **Dates:** January 4-8, 2025
- **Venue:** The Leela Palace, Bengaluru, India
- **Address:** The Leela Palace, 23, Kodihalli, Bangalore - 560008, Karnataka, India
- **Capacity:** 2400 attendees
- **Booking Status:** Available for booking, please contact for details.

## Marketing Strategies

### Social Media Campaign
1. **Platforms:** LinkedIn, Twitter, Facebook, Instagram
2. **Content Strategy:**
   - Share speaker highlights and profiles of dignitaries like Ashwini Vaishnaw and Chris Miller.
   - Post regular updates about conference topics including AI, Embedded Systems, and VLSI technology.
   - Use the hashtag **#VLSID2025** to boost visibility and track engagement.

### Email Marketing
- **Target Audience:** Engineers, students, industry professionals, and researchers.
- **Template:** Monthly newsletters containing event updates, registration details, and featured speakers.
- **Call to Action:** Encourage early registration with limited-time discount codes.

### Partnerships and Sponsorships
- Collaborate with key industry players such as NXP Semiconductors, Microchip Technology, and local universities for sponsorships and promotional support.
- Offer unique sponsorship packages that include speaking opportunities and booth setups.

## Attendee Engagement Activities

### Pre-Event Webinars
- Host live webinars with selected speakers to discuss trending topics in VLSI and Embedded Systems.
- Encourage audience interaction through Q&A sessions.

### Networking Opportunities
- Develop a mobile app for attendees to connect with each other, schedule meetings, and view the agenda.
- Organize mixer events to facilitate networking across academia and industry.

### Post-Event Content
- Create a recap video summarizing key moments from the conference.
- Share recorded sessions of keynote speeches and panel discussions to keep the conversation going.

## Conclusion
By implementing these marketing strategies and engagement activities, we aim to attract at least 500 potential attendees to the VLSI Design Conference 2025 and ensure a fruitful experience for all participants.
```